In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
movie_df = pd.read_csv("data/movie.csv")
movie_df

In [ ]:
rating_df = pd.read_csv("data/rating.csv")
rating_df = rating_df.drop("timestamp", axis=1)
rating_df

In [ ]:
num_of_movie = len(movie_df["title"].unique())
num_of_user = len(rating_df["userId"].unique())
num_of_rating = len(rating_df["movieId"].unique())

print(f"Num of movies: {num_of_movie}")
print(f"Num of ratings given by users: {num_of_user}")
print(f"Num of movie ratings: {num_of_rating}")

In [ ]:
movie_df.info()

In [ ]:
rating_df.info()

In [ ]:
movie_df.describe()

In [ ]:
movie_list = movie_df["title"].value_counts().keys()
count = movie_df["title"].value_counts()

movie_count = pd.DataFrame({"Movie-Title": movie_list, "Count": count}).reset_index(drop=True)
movie_count

In [ ]:
rating_list = rating_df["rating"].value_counts().keys()
count = rating_df["rating"].value_counts()

rating_count = pd.DataFrame({"Ratings": rating_list, "Count": count}).reset_index(drop=True)
rating_count

In [ ]:
sns.barplot(data=rating_count, x='Ratings', y='Count')
plt.show()

In [ ]:
movie_df.isnull().sum()

In [ ]:
rating_df.isnull().sum()

In [ ]:
movie_df.duplicated().sum()

In [ ]:
rating_df.duplicated().sum()